# ✅ Full Checker Demo

Run the complete orchestration pipeline end-to-end:
`plan -> parallel research -> judge`.

## 🔎 Sections
- ⚙️ Setup + auto-reload
- 🎛️ Runtime config
- 🗣️ Claim selection
- 🚀 Execute full checker
- 📦 Inspect structured artifacts
- 🧪 Quick tweaks

## ⚙️ Setup + auto-reload

In [2]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import os
import sys

ROOT = Path.cwd().resolve().parent if Path.cwd().name == "notebooks" else Path.cwd().resolve()
SRC = ROOT / "src"
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))

print(f"Root: {ROOT}")
print(f"Using src path: {SRC}")
print(f"OPENAI_API_KEY set: {bool(os.getenv('OPENAI_API_KEY'))}")
print(f"BRAVE_SEARCH_API_KEY set: {bool(os.getenv('BRAVE_SEARCH_API_KEY'))}")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Root: /Users/bertik/PhD/facticli
Using src path: /Users/bertik/PhD/facticli/src
OPENAI_API_KEY set: True
BRAVE_SEARCH_API_KEY set: False


## 🎛️ Runtime config

In [3]:
from facticli.orchestrator import FactCheckOrchestrator, OrchestratorConfig

model = os.getenv("FACTICLI_MODEL", "gpt-4.1-mini")
search_provider = os.getenv("FACTICLI_SEARCH_PROVIDER", "openai")
# search_provider = "brave"

config = OrchestratorConfig(
    model=model,
    max_checks=4,
    max_parallel_research=4,
    search_context_size="high",
    search_provider=search_provider,
)

if search_provider == "brave":
    assert os.getenv("BRAVE_SEARCH_API_KEY"), "Set BRAVE_SEARCH_API_KEY for brave mode."

orchestrator = FactCheckOrchestrator(config=config)
config

OrchestratorConfig(inference_provider='openai-agents', model='gpt-5.2', gemini_model='gemini-2.0-flash', max_turns=10, max_checks=4, max_parallel_research=4, search_context_size='high', search_provider='openai', search_results_per_query=5, max_search_queries_per_check=5, judge_extra_turns=2, research_timeout_seconds=120.0, research_retry_attempts=1)

## 🗣️ Claim selection

In [8]:
claim = "The Eiffel Tower was built in 1889 for the World's Fair."
# claim = "The first iPhone was released in 2008."
# claim = "Pluto is still classified as a planet by the IAU."
# claim = "The Amazon rainforest produces 20% of the world's oxygen."
claim = "Fialova vláda nastěhovala migranty do chat čechům."

claim

'Fialova vláda nastěhovala migranty do chat čechům.'

## 🚀 Execute full checker

In [9]:
from facticli.render import format_run_text

run = await orchestrator.check_claim(claim)
print(format_run_text(run, show_plan=True))

Claim
  Fialova vláda nastěhovala migranty do chat čechům.

Verdict
  Refuted (confidence: 0.78)

Justification
  Dohledané oficiální materiály vlády a Ministerstva vnitra k ubytování osob prchajících z Ukrajiny (2022) popisují uvolnění vlastních kapacit státu a samospráv (včetně jejich rekreačních zařízení) a smluvní/dobrovolné zajišťování ubytování s kompenzacemi. Nenalezeno žádné doložené rozhodnutí, které by nařizovalo či realizovalo nucené ubytování „migrantů“ v soukromých chatách českých občanů bez jejich souhlasu. Konkrétní mediálně popsané případy ubytování v chatkách/chalupách odpovídají spíše smluvnímu ubytování (např. kemp, dohodnutá chalupa), nikoli zabírání soukromého majetku.

Key Points
  - Vládní opatření z března 2022 mluví o uvolnění vlastních ubytovacích kapacit ministerstev/státu (ubytovny, rekreační zařízení, školicí střediska), ne o zabírání soukromých chat občanů.
  - MV ČR uvádí ubytování ve vlastních rekreačních objektech krajů/obcí a smluvní režim u soukromých

## 📦 Inspect structured artifacts

In [ ]:
run.plan.model_dump()

In [ ]:
run.report.model_dump()

## 🧪 Quick tweaks

- Switch between `openai` and `brave` search providers.
- Increase `max_checks` for broader decomposition.
- Compare outputs before/after editing prompt files in `src/facticli/prompts/`.